In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn as sk

In [3]:
ice_df = pd.read_csv('arrests-clean.csv')

In [4]:
ice_date = ice_df['apprehension_date'].tolist()
ice_citizen = ice_df['citizenship_country'].tolist()
ice_depart = ice_df['departure_country'].tolist()
daily_data = {'Date': [], 'Arrests': [], 'Deportations': [], 'Deportations to home country': [], 'Deportations elsewhere': [], 'Home country proportion': []}
daily_df = pd.DataFrame(data=daily_data)
# Keeps track of the dates put in daiy_data to prevent repeating rows
used_dates = []
# Contains the total number of arrests made for each day
ice_arrests = []
# Contains the total number of deportations that have happened
# that result from arrests made that day
ice_deport = []
# Contains the total of deportations that go to the arrested
# persons' country of citizenship
ice_home = []
# Contains the total of deportations going to other countries
ice_else = []
# The proportion of deportations that go to the correct country
ice_proportion = []

The convert_day method requires a date taken from the apprehension_date column of the dataset that is being processed. It checks that it has not previously processed data for this date before by checking a list of dates that it adds to whenever it processes an apprehension date for the first time. If it has not processed data relating to this date before, it finds the indices of arrests on that day in the cleaned ICE data. It then finds the total number of arrests, arrests leading to a deportation, the total numbers of arrests leading to deportations that either deport to the deported persons' country of citizenship or not, and the proportion of deportation-resulting arrests that go to the deported persons' coubtry of citizenship. It adds these to lists that will become columns of the daily_df dataframe afterwards.

In [5]:
from contextlib import nullcontext
# Takes all of the entries for each day
# and compresses them into a single row
def convert_day (apprehension_date):
  if not(apprehension_date in used_dates):
    used_dates.append(apprehension_date)
    index_list = []
    for index, date in enumerate(ice_date):
      if date == apprehension_date:
        index_list.append(index)
    citizen_list = []
    depart_list = []
    for index in index_list:
        citizen_list.append(ice_citizen[index])
        depart_list.append(ice_depart[index])
    home = 0
    apart = 0
    i = 0
    while i < len(index_list):
      if(not(pd.isnull(depart_list[i]))):
        if (citizen_list[i] == depart_list[i]):
          home += 1
        else:
          apart += 1
      i += 1
    ice_arrests.append(len(index_list))
    ice_deport.append(home+apart)
    ice_home.append(home)
    ice_else.append(apart)
    if home+apart>0:
      ice_proportion.append(home/(home+apart))
    else:
      ice_proportion.append(nullcontext)

In [6]:
for apprehension_date in ice_date:
  convert_day(apprehension_date)
# Move outcome to dataframe
daily_df['Date'] = used_dates
daily_df['Arrests'] = ice_arrests
daily_df['Deportations'] = ice_deport
daily_df['Deportations to home country'] = ice_home
daily_df['Deportations elsewhere'] = ice_else
daily_df['Home country proportion'] = ice_proportion


In [7]:
# Exports dataframe to csv
daily_df.to_csv('arrests_by_day.csv')